## Sweep bot

i'm fucking crashing out
why is nothing fucking working

In [1]:
%load_ext autoreload
%autoreload 2
# ^ make it so we can change inner files


In [2]:

import numpy as np
from pydrake.all import (
    AddDefaultVisualization,
    ContactModel,
    Demultiplexer,
    DiagramBuilder,
    InverseDynamicsController,
    MeshcatVisualizer,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    SceneGraph,
    Simulator,
    Sphere,
    StartMeshcat,
    VectorLogSink,
    VisualizationConfig,
)

from manipulation.meshcat_utils import (
    AddMeshcatTriad,
    PublishPositionTrajectory,
)
from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    Parser,
)
from scripts.load_scenario import load_scenario
from scripts.grasp_broom import plan_path, grasp_path, build_temp_plant
from scripts.meta_controller import MetaController

In [3]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7001


Click the link above to open Meshcat in your browser!


In [4]:
use_hydroelastic = True # this takes forever

def make_diagram():

    scenario = load_scenario(use_position=True)

    # since makehardwarestation finalizes the plant before giving it back to us
    def callback(parser: Parser):
        plant = parser.plant()
        plant.set_contact_model(ContactModel.kHydroelasticWithFallback)
        scene_graph: SceneGraph = parser.scene_graph()
        config = scene_graph.get_config()
        if use_hydroelastic:
            # one object rigid, all else compliant? table
            # HE modulus parameters
            # allow broom to turn more easily

            # add "handle" to broom to make it easier to grip?
            config.default_proximity_properties.compliance_type = "compliant"
        scene_graph.set_config(config)
        # AddDefaultVisualization(parser.builder(), meshcat)

    station = MakeHardwareStation(scenario, meshcat=meshcat, parser_prefinalize_callback=callback)
    plant: MultibodyPlant = station.plant()


    builder = DiagramBuilder()
    builder.AddSystem(station)

    to_point_cloud = AddPointClouds(
        scenario=scenario, station=station, builder=builder, meshcat=meshcat
    )
    builder.ExportOutput(
        to_point_cloud["camera0"].point_cloud_output_port(), "point_cloud0"
    )

    # we add diagram after building
    # really scuffed but i don't care at this point
    meta_controller_sys = MetaController(plant, station)
    meta_controller = builder.AddSystem(meta_controller_sys)

    """
    # ID CONTROLLER
    context = station.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    broom_pose=plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName('handle_link'))
    _, iiwa_plant, _ = build_temp_plant(broom_pose=broom_pose)
    ID_controller = builder.AddSystem(
        InverseDynamicsController(
            iiwa_plant,
            kp = np.ones(7) * 200,
            kd = np.ones(7) * 40,
            ki = np.zeros(7),
            has_reference_acceleration=False,
        )
    )

    control_estimated_state_input_port = ID_controller.get_input_port(0)
    control_desired_state_input_port = ID_controller.get_input_port(1)
    control_output_port = ID_controller.get_output_port(0)
    """

    demux = builder.AddSystem(Demultiplexer([7, 7, 1]))

    builder.Connect(
        meta_controller.get_output_port(),
        demux.get_input_port(),
    )

    # builder.Connect(
    #     demux.get_output_port(0),
    #     control_desired_state_input_port
    # )

    builder.Connect(
        demux.get_output_port(0),
        station.GetInputPort("iiwa.position")
    )
    builder.ExportOutput(demux.get_output_port(0), 'pos')
    builder.ExportOutput(demux.get_output_port(1), 'vel')

    builder.Connect(
        demux.get_output_port(2), 
        station.GetInputPort("wsg.position")
    )

    # builder.Connect(
    #     station.GetOutputPort("iiwa_state"),
    #     control_estimated_state_input_port
    # )

    # builder.Connect(
    #     control_output_port,
    #     station.GetInputPort("iiwa_actuation")
    # )
    # AddDefaultVisualization(builder, meshcat)

    diagram = builder.Build()
    # meta_controller_sys.add_diagram(diagram)

    # for debugging
    meta_controller_sys.meshcat = meshcat

    return diagram, plant, station, meta_controller_sys

# diagram = builder.Build()

In [5]:
diagram, plant, station, controller = make_diagram()

In [ ]:
from scripts.trajectory_helpers import MoveToPregrip

simulator = Simulator(diagram)
context = simulator.get_mutable_context()
plant_context = plant.GetMyContextFromRoot(context)
controller.add_diagram(diagram, context, meshcat)
simulator.set_target_realtime_rate(1.0)
meshcat.StartRecording()
simulator.Initialize()
end_time = 3.4
time = 0
iiwa = plant.GetModelInstanceByName('iiwa')
pos_port = simulator.get_system().GetOutputPort('pos')
# pant.SetPOsitions(pant_context, trajectory.value(t))
#                 visaulizer.ForcedPublish(visaulzier_context)
traj_gen = MoveToPregrip(meshcat)
trajectory, _ = traj_gen.trajectory(controller)
"""
while time < end_time:
    time += 0.1
    plant.SetPositions()
    plant.SetPositions(plant_context, iiwa, pos_port.Eval(context))
    simulator.AdvanceTo(time)
"""
# simulator.AdvanceTo(3.4)
while time < end_time:
    time += 0.05
    q = trajectory.value(time)
    plant.SetPositions(plant_context, iiwa, q)
    diagram.ForcedPublish(context)
    # time.sleep(0.05)

meshcat.StopRecording()
meshcat.PublishRecording()
# controller.sub_meshcat.StopRecording()
# controller.sub_meshcat.PublishRecording()

[[-0.6200788  -0.618094   -0.61610925 ... -0.51885533 -0.51687056
  -0.5148858 ]
 [ 1.0237491   1.0237491   1.0237491  ...  0.5176319   0.5176319
   0.5176319 ]
 [ 0.0499999   0.0499999   0.0499999  ...  0.0499999   0.0499999
   0.0499999 ]]
trying angle -2.158798685643362
broom pose RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.6, 1.4, 0.0],
)
7
(7, 10)
Solver failed
['path position constraint at s=1[0]: 2.999999 <= 2.997564 <= 3.000000']
attempts needed: 1


In [7]:
%tb

No traceback available to show.
